In [163]:
import os
import pandas as pd

os.chdir(r'C:\Users\pasht\OneDrive\Документы\4 курс 1 семестр\визуализация данных\практика 4')
online_ecom = pd.read_csv('Online ecom.csv', sep=';')
online_ecom[:3]

,CID,TID,Gender,Age Group,Purchase Date,Product Category,Discount Availed,Discount Name,Discount Amount (INR),Gross Amount,Net Amount,Purchase Method,Location
0,943146,5876328741,Female,25-45,30.08.2023 20:27,Electronics,Yes,FESTIVE50,64.30,725.304000,661.004000,Credit Card,Ahmedabad
1,180079,1018503182,Male,25-45,23.02.2024 9:33,Electronics,Yes,SEASONALOFFER21,175.19,4638.991875,4463.801875,Credit Card,Bangalore
2,337580,3814082218,Female,60 and above,06.03.2022 9:09,Clothing,Yes,SEASONALOFFER21,211.54,1986.372575,1774.832575,Credit Card,Delhi


In [165]:
#видим что Purchase Date не в формате datetime
online_ecom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55000 entries, 0 to 54999
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   CID                    55000 non-null  int64  
 1   TID                    55000 non-null  int64  
 2   Gender                 55000 non-null  object 
 3   Age Group              55000 non-null  object 
 4   Purchase Date          55000 non-null  object 
 5   Product Category       55000 non-null  object 
 6   Discount Availed       55000 non-null  object 
 7   Discount Name          27415 non-null  object 
 8   Discount Amount (INR)  55000 non-null  float64
 9   Gross Amount           55000 non-null  float64
 10  Net Amount             55000 non-null  float64
 11  Purchase Method        55000 non-null  object 
 12  Location               55000 non-null  object 
dtypes: float64(3), int64(2), object(8)
memory usage: 5.5+ MB


In [167]:
#прератим Purchase Date в формат datetime используюя normalize для обнуления времени, чтобы оставить только саму дату
online_ecom['Purchase Date']=pd.to_datetime(online_ecom['Purchase Date'], format = '%d.%m.%Y %H:%M').dt.normalize()
online_ecom.info()
online_ecom['Purchase Date'][:3]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55000 entries, 0 to 54999
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   CID                    55000 non-null  int64         
 1   TID                    55000 non-null  int64         
 2   Gender                 55000 non-null  object        
 3   Age Group              55000 non-null  object        
 4   Purchase Date          55000 non-null  datetime64[ns]
 5   Product Category       55000 non-null  object        
 6   Discount Availed       55000 non-null  object        
 7   Discount Name          27415 non-null  object        
 8   Discount Amount (INR)  55000 non-null  float64       
 9   Gross Amount           55000 non-null  float64       
 10  Net Amount             55000 non-null  float64       
 11  Purchase Method        55000 non-null  object        
 12  Location               55000 non-null  object        
dtypes

0   2023-08-30
1   2024-02-23
2   2022-03-06
Name: Purchase Date, dtype: datetime64[ns]

In [331]:
online_ecom['count_of_purchase'] = online_ecom['CID']
RFM = online_ecom.groupby('CID').agg({'count_of_purchase':'count', 'Purchase Date':'max', 'Net Amount':'sum'})
RFM.rename(columns = {'Purchase Date':'last_purchase', 'Net Amount':'total_amount'}, inplace = True)
RFM.sort_values(by = 'last_purchase', ascending = False, inplace = True)
RFM[:3]

,count_of_purchase,last_purchase,total_amount
CID,,,
401437,2,2024-09-18,1916.20950
871440,2,2024-09-18,3375.73975
116470,1,2024-09-18,3495.90000


In [333]:
RFM['last_purchase']=pd.to_datetime(RFM['last_purchase'], format = '%d.%m.%Y')
RFM.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29071 entries, 401437 to 478035
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   count_of_purchase  29071 non-null  int64         
 1   last_purchase      29071 non-null  datetime64[ns]
 2   total_amount       29071 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 908.5 KB


In [335]:
RFM['max_date'] = RFM['last_purchase'].max()
RFM['days_since_last_order'] = RFM['max_date']-RFM['last_purchase']
RFM['days_since_last_order'] = RFM['days_since_last_order'].dt.days
RFM[:3]

,count_of_purchase,last_purchase,total_amount,max_date,days_since_last_order
CID,,,,,
401437,2,2024-09-18,1916.20950,2024-09-18,0
871440,2,2024-09-18,3375.73975,2024-09-18,0
116470,1,2024-09-18,3495.90000,2024-09-18,0


In [337]:
q1 = RFM['days_since_last_order'].quantile(0.25)
q3 = RFM['days_since_last_order'].quantile(0.75)

#сделаем через loc чтобы вспомнить loc, удобнее конечно делать через qcut()
RFM.loc[RFM['days_since_last_order']<=q1, 'days_since_last_order']=3
RFM.loc[(RFM['days_since_last_order']>q1) & (RFM['days_since_last_order']<q3), 'days_since_last_order']=2
RFM.loc[RFM['days_since_last_order']>=q3, 'days_since_last_order']=1
RFM.rename(columns = {'days_since_last_order':'Recency'}, inplace = True)
RFM[:3]

,count_of_purchase,last_purchase,total_amount,max_date,Recency
CID,,,,,
401437,2,2024-09-18,1916.20950,2024-09-18,3
871440,2,2024-09-18,3375.73975,2024-09-18,3
116470,1,2024-09-18,3495.90000,2024-09-18,3


In [339]:
#остальные разобьем через cut и qcut 
RFM['Frequency'] = pd.cut(RFM['count_of_purchase'],bins=[0, 2, 6, 9],labels=[3, 2, 1])
RFM['Monetary'] = pd.qcut(RFM.total_amount, q = [0,0.25,0.75,1], labels = [3,2,1])
RFM[:3]


,count_of_purchase,last_purchase,total_amount,max_date,Recency,Frequency,Monetary
CID,,,,,,,
401437,2,2024-09-18,1916.20950,2024-09-18,3,3,3
871440,2,2024-09-18,3375.73975,2024-09-18,3,3,2
116470,1,2024-09-18,3495.90000,2024-09-18,3,3,2


In [341]:
display(RFM['Recency'].value_counts(),
        RFM['Frequency'].value_counts(),
        RFM['Monetary'].value_counts())

Recency
2    14508
3     7285
1     7278
Name: count, dtype: int64

Frequency
3    22388
2     6636
1       47
Name: count, dtype: int64

Monetary
2    14535
3     7268
1     7268
Name: count, dtype: int64

## Вывод: Т.к анализируемые данные имеют лежат в интарвале времени 5 лет, а кол-во клиентов совершающих от 6 и более заказов крайне низкое, компания предоставляет услуги с низкой частотой потребления